In [80]:
import pandas as pd
df = pd.read_csv('data/frequent_flyers.csv',sep=',')

In [81]:
personal_data = df[['NickName', 'Name', 'FB number','KE number','SU number','DT number']]
personal_data['flights'] = 0
personal_data['country_mismatches'] = 0
personal_data = personal_data.drop_duplicates()

C:\Users\Герман\AppData\Local\Temp\ipykernel_6340\1350350623.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  personal_data['flights'] = 0
C:\Users\Герман\AppData\Local\Temp\ipykernel_6340\1350350623.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  personal_data['country_mismatches'] = 0


In [82]:
country_mismatches = df[['NickName','Date', 'Arrival country', 'Departure country']]
for index,pers in personal_data.iterrows():
    pers_flights = country_mismatches[country_mismatches['NickName'] == pers['NickName']].sort_values(by=['Date'])
    is_first = True
    last_country = None
    mismatch_count = 0
    for i, flight in pers_flights.iterrows():
        if (is_first == True):
            current_country = flight['Arrival country']
            is_first = False
            continue
        if (current_country != flight['Departure country']):
            mismatch_count += 1
        current_country = flight['Arrival country']
    personal_data.at[index,'country_mismatches'] = mismatch_count
    personal_data.at[index,'flights'] = len(pers_flights)

In [95]:
bonus = pd.read_csv('data/bonus.csv',sep=',')
for i,row in bonus.iterrows():
    names = row['name'].split(' ')
    bonus.at[i,'name'] = names[1] +' '+names[0]

In [129]:
bonus_FB = bonus[bonus['bonus'] == 'FB']
bonus_FB = bonus_FB.drop(['id','bonus'], axis=1)
bonus_DT = bonus[bonus['bonus'] == 'DT']
bonus_DT = bonus_DT.drop(['id','bonus'], axis=1)
bonus_KE = bonus[bonus['bonus'] == 'KE']
bonus_KE = bonus_KE.drop(['id','bonus'], axis=1)
bonus_SU = bonus[bonus['bonus'] == 'SU']
bonus_SU = bonus_SU.drop(['id','bonus'], axis=1)
personal_data.rename(columns={'FB number': 'FB', 'DT number': 'DT','KE number': 'KE','SU number': 'SU'}, inplace=True)
personal_data

,NickName,Name,FB,KE,SU,DT,flights,country_mismatches
0,FrequentFlyer0,NaN,NaN,889215424.0,NaN,484697244.0,22,0
1,FrequentFlyer2,ZYKOV NAZAR,NaN,388423193.0,658121632.0,356246372.0,20,1
2,FrequentFlyer4,NaN,NaN,NaN,224950608.0,NaN,18,0
3,FrequentFlyer5,NaN,436692325.0,NaN,NaN,NaN,22,0
4,FrequentFlyer6,D'YAKOV RUSLAN,629903943.0,629925275.0,525500144.0,479114128.0,21,0
...,...,...,...,...,...,...,...,...
66617,FrequentFlyer99992,NaN,NaN,NaN,401792585.0,NaN,13,0
66618,FrequentFlyer99993,NaN,NaN,NaN,751322904.0,NaN,25,0
66619,FrequentFlyer99995,BONDAREVA STEFANIYA,18721601.0,125809576.0,NaN,406967834.0,24,0
66620,FrequentFlyer99996,NaN,183116620.0,275536799.0,449778932.0,702680548.0,16,0


In [170]:
#получаем имена из файла ???
personal_data = personal_data[['NickName',	'Name',	'FB',	'KE',	'SU',	'DT',	'flights',	'country_mismatches']]
personal_data = pd.merge(personal_data, bonus_FB, left_on='FB',right_on='number', how='left')
personal_data = pd.merge(personal_data, bonus_KE, left_on='KE',right_on='number', how='left', suffixes=('_left1', '_right1'))
personal_data = pd.merge(personal_data, bonus_SU, left_on='SU',right_on='number', how='left', suffixes=('_left2', '_right2'))
personal_data = pd.merge(personal_data, bonus_DT, left_on='DT',right_on='number',how='left', suffixes=('_left3', '_right3'))
personal_data = personal_data.drop(['number_left1',	'number_right1', 'number_left3', 'number_right3'], axis=1)
personal_data['name_left1'] = personal_data['name_left1'].fillna(personal_data['name_right1'])
personal_data['name_left1'] = personal_data['name_left1'].fillna(personal_data['name_left3'])
personal_data['name_left1'] = personal_data['name_left1'].fillna(personal_data['name_right3'])
personal_data = personal_data.drop(['name_right1',	'name_left3',	'name_right3'],axis=1)
personal_data.rename(columns={'name_left1': 'bonus name'}, inplace=True)

personal_data.drop_duplicates(subset='NickName', inplace=True)

#удаляем строки где нет никаких имен
named_data = personal_data.dropna(subset=['Name', 'bonus name'], how='all')

In [171]:
import difflib
def are_similar(str1, str2, threshold=0.7):
    similarity_ratio = difflib.SequenceMatcher(None, str1, str2).ratio()
    return similarity_ratio >= threshold

In [276]:
#ищем несоответствие имен
selected_rows = []
for i, row in names_mismatch.iterrows():
    if are_similar(row['Name'], row['bonus name']) == False:
        selected_rows.append(row)
names_mismatch = pd.DataFrame(selected_rows)
names_mismatch

,NickName,Name,FB,KE,SU,DT,flights,country_mismatches,bonus name
47864,FrequentFlyer71993,GOLOVINA OLESYA,139888197.0,757384867.0,446648304.0,298868326.0,22,0,MASLENNIKOV ROMAN


In [173]:
# объединяем поля с именами
named_data['Name'] = named_data['Name'].fillna(named_data['bonus name'])
named_data = named_data.drop('bonus name', axis=1)

C:\Users\Герман\AppData\Local\Temp\ipykernel_6340\2175298102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  named_data['Name'] = named_data['Name'].fillna(named_data['bonus name'])


In [175]:
named_data['susp percent'] = named_data['country_mismatches'] * 100 / named_data['flights']

In [191]:
#накладываем ограничение на минимальное число полетов
num_flights_filter= named_data[named_data['flights'] >=10]

In [205]:
for i in range(0,24,3):
    print('from',i, 'to', i+3)
    print(num_flights_filter[num_flights_filter['susp percent'] < i+3].shape[0] - num_flights_filter[num_flights_filter['susp percent'] <= i].shape[0])

from 0 to 3
12
from 3 to 6
2523
from 6 to 9
535
from 9 to 12
157
from 12 to 15
31
from 15 to 18
21
from 18 to 21
5
from 21 to 24
4


In [209]:
#установим границу для потенциальных шпионов в 12% перелетов со сменой страны
many_susp_flights = num_flights_filter[num_flights_filter['susp percent'] >= 12]

In [280]:
many_susp_flights

,NickName,Name,FB,KE,SU,DT,flights,country_mismatches,susp percent
1344,FrequentFlyer1974,VASIL'EV MIKHAIL,NaN,413319294.0,NaN,NaN,21,3,14.285714
1804,FrequentFlyer2673,TSAREVA ALEXANDRA,744923811.0,127460365.0,NaN,663425418.0,16,2,12.500000
2029,FrequentFlyer3005,TUMANOVA VERONIKA,NaN,NaN,NaN,220199273.0,24,4,16.666667
3537,FrequentFlyer5256,KUPRIYANOVA SOFIYA,460168399.0,NaN,NaN,NaN,13,2,15.384615
10476,FrequentFlyer15704,TRET'YAKOVA ALENA,NaN,NaN,NaN,419131661.0,22,3,13.636364
...,...,...,...,...,...,...,...,...,...
62414,FrequentFlyer93776,PARAMONOV ROMAN,141122585.0,NaN,NaN,NaN,14,2,14.285714
63889,FrequentFlyer96016,DUBININA TAT'YANA,999218066.0,436726511.0,NaN,172965739.0,20,3,15.000000
64300,FrequentFlyer96626,VOLKOV ALEXEY,NaN,50336298.0,NaN,851416082.0,21,3,14.285714
64390,FrequentFlyer96748,ZAKHAROVA ALLA,NaN,NaN,NaN,886670182.0,15,2,13.333333


In [287]:
named_data[['Name', 'flights','country_mismatches','susp percent']].to_csv('data/hysto_data.csv', index=False, header=True)

Поиск несостыковок

In [246]:
fdf = pd.read_csv('data/Skyteam_Timetable.csv',sep=',')

In [247]:
flights = fdf[['From_Code', 'To_Code', 'Validity', 'Flight']]
people = df[['NickName','Departure port','Arrival port', 'Date', 'Flight']]
people = pd.merge(people, named_data, on='NickName', how='right')
people = people[['Name','Departure port','Arrival port', 'Date', 'Flight']]
people

,Name,Departure port,Arrival port,Date,Flight
0,ZYKOV NAZAR,PVG,NRT,2017-08-31,MU8705
1,ZYKOV NAZAR,SIN,TPE,2017-08-11,GA9995
2,ZYKOV NAZAR,TPE,NRT,2017-08-23,CI9902
3,ZYKOV NAZAR,PKC,VVO,2017-01-04,SU5615
4,ZYKOV NAZAR,AQG,CAN,2017-10-17,CZ3320
...,...,...,...,...,...
935834,ISAEV AYDAR,IAD,JFK,2017-12-15,DL3345
935835,ISAEV AYDAR,SVO,PRG,2017-08-12,SU2014
935836,ISAEV AYDAR,ATL,LGA,2017-12-29,DL2463
935837,ISAEV AYDAR,OVB,SVO,2017-06-02,SU1463


In [248]:
#предположим что у нас данные об авиарейсах за 2017 год, так что удалим все полеты людей за 2018
select2017 = []
for i, row in people.iterrows():
    if ('8' == row['Date'][3:4]) == False:
        select2017.append(row)
people17 = pd.DataFrame(select2017)

In [249]:
from datetime import datetime

# Функция для преобразования месяца из строкового формата в числовой
def month_str_to_num(month_str):
    month_num = datetime.strptime(month_str, '%b').month
    return str(month_num)

In [250]:
#преобразуем дату в нужный формат
for i,row in flights.iterrows():
    flights.at[i, 'Date'] = '2017-' + month_str_to_num(row['Validity'][3:6]) +'-'+ row['Validity'][:2]
flights.drop('Validity',axis=1, inplace=True)

C:\Users\Герман\AppData\Local\Temp\ipykernel_6340\3419909204.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights.at[i, 'Date'] = '2017-' + month_str_to_num(row['Validity'][3:6]) +'-'+ row['Validity'][:2]
C:\Users\Герман\AppData\Local\Temp\ipykernel_6340\3419909204.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights.drop('Validity',axis=1, inplace=True)


In [267]:
mismatch_flights = []
for i,row in people17.iterrows():
    route = flights[flights['Flight'] == row['Flight']]
    date = route[route['Date'] == row['Date']]
    if (date.empty):
        continue
    departure = date[date['From_Code'] == row['Departure port']]
    if (departure.empty):
        #mismatch_flights.append(row)
        continue
    if (departure.iloc[0]['To_Code'] != row['Arrival port']):
        mismatch_flights.append(row)

KeyboardInterrupt: 

In [270]:
spies2 = pd.DataFrame(mismatch_flights)
spies2 = spies2.drop_duplicates(subset='Name')
spies2

,Name,Departure port,Arrival port,Date,Flight
2509,CHERNYKH BOGDAN,JFK,MIA,2017-11-27,DL2796
3293,BELIAEV SAVVA,PEK,SHA,2017-11-01,MU5693
4096,AGAFONOV EGOR,SLC,SAT,2017-12-25,DL3697
6002,CHEBOTAREVA ALISA,ATL,EYW,2017-12-22,DL3424
10793,MATVEEV IL'YA,BUF,DTW,2017-11-20,DL6004
...,...,...,...,...,...
330001,BASOVA MADINA,LGA,BTV,2017-11-10,DL3490
331034,ZUBOVA LIUDMILA,CAN,NDG,2017-11-03,CZ6437
331370,KORSHUNOVA EVA,ATL,ABQ,2017-11-01,DL972
332201,DAVYDOVA VIKTORIYA,FOC,TNA,2017-11-01,MF8041


In [284]:
#oбъединяем всех найденных шпионов и избавляемся от дупликатов
combined = pd.concat([names_mismatch['Name'], many_susp_flights['Name'],spies2['Name']], ignore_index=True)
combined.drop_duplicates()

0         GOLOVINA OLESYA
1        VASIL'EV MIKHAIL
2       TSAREVA ALEXANDRA
3       TUMANOVA VERONIKA
4      KUPRIYANOVA SOFIYA
              ...        
285         BASOVA MADINA
286       ZUBOVA LIUDMILA
287        KORSHUNOVA EVA
288    DAVYDOVA VIKTORIYA
289         MALININ SEMEN
Name: Name, Length: 290, dtype: object

In [286]:
#файл с возможными шпионами
combined.to_csv('data/frequent_spies.csv', index=False, header=True)